# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Nov 10 2022 10:04AM,250477,22,11102022.2,FUJIFILM Diosynth Biotechnologies Texas LLC,Released
1,Nov 10 2022 10:00AM,250475,15,8751391,FUJIFILM Diosynth Biotechnologies Texas LLC,Released
2,Nov 10 2022 9:57AM,250474,19,0000102406,Paragon BioTeck,Executing
3,Nov 10 2022 9:55AM,250473,16,8758268,Sartorius Stedim North America,Released
4,Nov 10 2022 9:44AM,250471,19,ACG-2102491829,ACG North America LLC,Released
5,Nov 10 2022 9:37AM,250470,10,CTF0011500,"Citieffe, Inc.",Released
6,Nov 10 2022 9:37AM,250470,10,CTF0011501,"Citieffe, Inc.",Released
7,Nov 10 2022 9:37AM,250470,10,8758260,"Citieffe, Inc.",Released
8,Nov 10 2022 9:20AM,250469,10,PRF-41520,Bio-PRF,Released
9,Nov 10 2022 9:18AM,250468,10,PRF-41516,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
14,250471,Released,1
15,250473,Released,1
16,250474,Executing,1
17,250475,Released,1
18,250477,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
250471,NaN,1.0
250473,NaN,1.0
250474,1.0,NaN
250475,NaN,1.0
250477,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
250390,1.0,0.0
250391,0.0,1.0
250416,0.0,83.0
250436,0.0,1.0
250437,0.0,6.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
250390,1,0
250391,0,1
250416,0,83
250436,0,1
250437,0,6


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,250390,1,0
1,250391,0,1
2,250416,0,83
3,250436,0,1
4,250437,0,6


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,250390,1,
1,250391,,1
2,250416,,83
3,250436,,1
4,250437,,6


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Nov 10 2022 10:04AM,250477,22,FUJIFILM Diosynth Biotechnologies Texas LLC
1,Nov 10 2022 10:00AM,250475,15,FUJIFILM Diosynth Biotechnologies Texas LLC
2,Nov 10 2022 9:57AM,250474,19,Paragon BioTeck
3,Nov 10 2022 9:55AM,250473,16,Sartorius Stedim North America
4,Nov 10 2022 9:44AM,250471,19,ACG North America LLC
5,Nov 10 2022 9:37AM,250470,10,"Citieffe, Inc."
8,Nov 10 2022 9:20AM,250469,10,Bio-PRF
9,Nov 10 2022 9:18AM,250468,10,Bio-PRF
10,Nov 10 2022 8:58AM,250465,16,Sartorius Bioprocess Solutions
14,Nov 10 2022 8:58AM,250466,16,Sartorius Lab Products and Service


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Nov 10 2022 10:04AM,250477,22,FUJIFILM Diosynth Biotechnologies Texas LLC,,1
1,Nov 10 2022 10:00AM,250475,15,FUJIFILM Diosynth Biotechnologies Texas LLC,,1
2,Nov 10 2022 9:57AM,250474,19,Paragon BioTeck,1,
3,Nov 10 2022 9:55AM,250473,16,Sartorius Stedim North America,,1
4,Nov 10 2022 9:44AM,250471,19,ACG North America LLC,,1
5,Nov 10 2022 9:37AM,250470,10,"Citieffe, Inc.",,3
6,Nov 10 2022 9:20AM,250469,10,Bio-PRF,,1
7,Nov 10 2022 9:18AM,250468,10,Bio-PRF,,1
8,Nov 10 2022 8:58AM,250465,16,Sartorius Bioprocess Solutions,,4
9,Nov 10 2022 8:58AM,250466,16,Sartorius Lab Products and Service,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 10 2022 10:04AM,250477,22,FUJIFILM Diosynth Biotechnologies Texas LLC,1,
1,Nov 10 2022 10:00AM,250475,15,FUJIFILM Diosynth Biotechnologies Texas LLC,1,
2,Nov 10 2022 9:57AM,250474,19,Paragon BioTeck,,1
3,Nov 10 2022 9:55AM,250473,16,Sartorius Stedim North America,1,
4,Nov 10 2022 9:44AM,250471,19,ACG North America LLC,1,
5,Nov 10 2022 9:37AM,250470,10,"Citieffe, Inc.",3,
6,Nov 10 2022 9:20AM,250469,10,Bio-PRF,1,
7,Nov 10 2022 9:18AM,250468,10,Bio-PRF,1,
8,Nov 10 2022 8:58AM,250465,16,Sartorius Bioprocess Solutions,4,
9,Nov 10 2022 8:58AM,250466,16,Sartorius Lab Products and Service,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 10 2022 10:04AM,250477,22,FUJIFILM Diosynth Biotechnologies Texas LLC,1,
1,Nov 10 2022 10:00AM,250475,15,FUJIFILM Diosynth Biotechnologies Texas LLC,1,
2,Nov 10 2022 9:57AM,250474,19,Paragon BioTeck,,1
3,Nov 10 2022 9:55AM,250473,16,Sartorius Stedim North America,1,
4,Nov 10 2022 9:44AM,250471,19,ACG North America LLC,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Nov 10 2022 10:04AM,250477,22,FUJIFILM Diosynth Biotechnologies Texas LLC,1.0,NaN
1,Nov 10 2022 10:00AM,250475,15,FUJIFILM Diosynth Biotechnologies Texas LLC,1.0,NaN
2,Nov 10 2022 9:57AM,250474,19,Paragon BioTeck,NaN,1.0
3,Nov 10 2022 9:55AM,250473,16,Sartorius Stedim North America,1.0,NaN
4,Nov 10 2022 9:44AM,250471,19,ACG North America LLC,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 10 2022 10:04AM,250477,22,FUJIFILM Diosynth Biotechnologies Texas LLC,1.0,0.0
1,Nov 10 2022 10:00AM,250475,15,FUJIFILM Diosynth Biotechnologies Texas LLC,1.0,0.0
2,Nov 10 2022 9:57AM,250474,19,Paragon BioTeck,0.0,1.0
3,Nov 10 2022 9:55AM,250473,16,Sartorius Stedim North America,1.0,0.0
4,Nov 10 2022 9:44AM,250471,19,ACG North America LLC,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1753184,24.0,11.0
15,500891,84.0,0.0
16,1001863,7.0,0.0
19,500945,1.0,1.0
21,500781,1.0,1.0
22,250477,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1753184,24.0,11.0
1,15,500891,84.0,0.0
2,16,1001863,7.0,0.0
3,19,500945,1.0,1.0
4,21,500781,1.0,1.0
5,22,250477,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,24.0,11.0
1,15,84.0,0.0
2,16,7.0,0.0
3,19,1.0,1.0
4,21,1.0,1.0
5,22,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,24.0
1,15,Released,84.0
2,16,Released,7.0
3,19,Released,1.0
4,21,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,21,22
Status,,,,,,
Executing,11.0,0.0,0.0,1.0,1.0,0.0
Released,24.0,84.0,7.0,1.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,21,22
0,Executing,11.0,0.0,0.0,1.0,1.0,0.0
1,Released,24.0,84.0,7.0,1.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,21,22
0,Executing,11.0,0.0,0.0,1.0,1.0,0.0
1,Released,24.0,84.0,7.0,1.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()